In [1]:
pip install pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 682.7 kB/s eta 0:00:01
     ------------ ------------------------- 20.5/61.2 kB 682.7 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 281.8 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 281.8 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 272.7 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requir

In [2]:
from selenium import webdriver
driver = webdriver.Chrome()

In [4]:
driver.get(f"https://ko.dict.naver.com/#/search?range=example&page=1&query={'아침'}&autoConvert=")

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pyautogui
# keyword = pyautogui.prompt("검색어를 입력하세요.")
# pageNum = 1
# lastpage = pyautogui.prompt("마지막 페이지를 입력하세요.")
# for i in range(1, int(lastpage) * 10, 10):
#     print(f"{pageNum}페이지입니다======")
#     response = requests.get(f"https://ko.dict.naver.com/#/search?query={keyword}}&range=example")
#     html = response.text
#     soup = BeautifulSoup(html, 'html.parser')
#     links = soup.select(".news_tit") #결과 리스트
#     for link in links:
#         title = link.text
#         url = link.attrs['href']
#         print(title, url)
#     pageNum = pageNum + 1

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# Chrome WebDriver의 경로를 지정
#chrome_driver_path = '/path/to/chromedriver'  # 본인의 환경에 맞게 수정

# 여러 개의 키워드 리스트
keywords = ["아침", "점심", "저녁", "오전", "오후", "밤"]

# Chrome WebDriver 설정
#driver = webdriver.Chrome(chrome_driver_path)

for keyword in keywords:
    pageNum = 1
    lastpage = input("마지막 페이지를 입력하세요: ")

    # 네이버 사전 페이지로 이동
    driver.get(f"https://ko.dict.naver.com/#/search?range=example&page=1&query={keyword}&autoConvert=")

    for i in range(1, int(lastpage) * 10, 10):
        print(f"{pageNum}페이지입니다======")

        # 페이지 로딩을 기다림 (3초 기다리고 조절 가능)
        time.sleep(3)

        # 검색 결과를 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        links = soup.select(".news_tit")  # 결과 리스트
        for link in links:
            title = link.text
            url = link.attrs['href']
            print(title, url)

        # 다음 페이지로 이동
        try:
            next_button = driver.find_element_by_css_selector(".pagination_next")
            next_button.click()
        except NoSuchElementException:
            break

        pageNum += 1

# WebDriver 종료
driver.quit()


In [15]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.by import By

# 여러 개의 키워드 리스트
keywords = ["아침", "점심", "저녁", "오전", "오후", "밤"]

# Chrome WebDriver 설정
driver = webdriver.Chrome()

# 수집된 데이터를 저장할 리스트
dataframes = []

for keyword in keywords:
    # 해당 키워드의 데이터프레임 초기화
    keyword_data = []

    # 1페이지부터 1페이지까지 크롤링
    for pageNum in range(1, 2):
        print(f"{keyword} 검색 {pageNum}페이지입니다======")

        # 네이버 사전 페이지로 이동
        driver.get(f"https://ko.dict.naver.com/#/search?range=example&page={pageNum}&query={keyword}&autoConvert=")

        # 페이지 로딩을 기다림 (3초 기다리고 조절 가능)
        time.sleep(3)
        # # 현재 페이지의 소스 코드
        # current_page_source = driver.page_source

        # # 현재 페이지와 이전 페이지의 소스 코드가 같다면 마지막 페이지까지 크롤링한 것으로 간주
        # if current_page_source == prev_page_source:
        #     break

        # 검색 결과를 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        examples = soup.select('.origin[lang="ko"] .text')

        for example in examples:
            title = example.text.strip()
            url = example.find_next('a', class_='link_entry')['href']
            keyword_data.append({'Title': title, 'URL': url})

        # 다음 페이지로 이동
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
            next_button.click()
        except NoSuchElementException:
            break

    # 수집된 데이터를 데이터프레임으로 변환하여 리스트에 추가
    keyword_df = pd.DataFrame(keyword_data)
    dataframes.append(keyword_df)

# WebDriver 종료
driver.quit()

# 키워드별로 수집된 데이터프레임을 합치기
result_df = pd.concat(dataframes, keys=keywords, names=['Keyword'])

# 결과 데이터프레임 출력
print(result_df)


아침 검색 1페이지입니다======
점심 검색 1페이지입니다======
저녁 검색 1페이지입니다======
오전 검색 1페이지입니다======
오후 검색 1페이지입니다======
밤 검색 1페이지입니다======
                                                        Title  \
Keyword                                                         
아침      0                                  그는 아침 일찍 일어나는 편이다.   
        1                                  그는 아침을 거른 채 출근하였다.   
        2                               아침을 설치고 왔더니 참말로 시장하오.   
        3            모처럼 휴일을 맞은 젊은이들은 이른 아침부터 극장 앞에 줄을 서 있었다.   
        4       아치골댁은…부뚜막에 걸터앉아 아침 겸 점심인 고구마 밥을 허기지게 먹기 시작했다.   
...                                                       ...   
밤       10                            새언니는 또 얼마나 찐 밤을 좋아했었는지.   
        11                  아이들이 떼를 지어서 밤을 줍고 상수리를 털기에 부산하였다.   
        12         이럴 때 피부 트러블을 완화시킬 수 있는 밤을 직접 만들어 보는 건 어떨까?   
        13        결국 군사들은 밤을 도와 다 흩어져 버리고 수령들도 모두 각각 도망해 버렸다.   
        14  체면상 싫다는 소리는 차마 못 하고 새우등으로 밤을 지내고는 다음 날로 당장 섬을 ...   

                                   

In [16]:
result_df

Title  \
Keyword                                                         
아침      0                                  그는 아침 일찍 일어나는 편이다.   
        1                                  그는 아침을 거른 채 출근하였다.   
        2                               아침을 설치고 왔더니 참말로 시장하오.   
        3            모처럼 휴일을 맞은 젊은이들은 이른 아침부터 극장 앞에 줄을 서 있었다.   
        4       아치골댁은…부뚜막에 걸터앉아 아침 겸 점심인 고구마 밥을 허기지게 먹기 시작했다.   
...                                                       ...   
밤       10                            새언니는 또 얼마나 찐 밤을 좋아했었는지.   
        11                  아이들이 떼를 지어서 밤을 줍고 상수리를 털기에 부산하였다.   
        12         이럴 때 피부 트러블을 완화시킬 수 있는 밤을 직접 만들어 보는 건 어떨까?   
        13        결국 군사들은 밤을 도와 다 흩어져 버리고 수령들도 모두 각각 도망해 버렸다.   
        14  체면상 싫다는 소리는 차마 못 하고 새우등으로 밤을 지내고는 다음 날로 당장 섬을 ...   

                                                      URL  
Keyword                                                    
아침      0   #/entry/koko/87f6c894a9814729bdf2af27104da93c  
        1   #/entry/koko/87f6c894a9814729bdf2af27104da93c  
        2   #/entry/koko/87f6c894a9814729bdf2af27104da93c  
        3   #/entry/koko/87f6c894a9814729bdf2af27104da93c  
        4   #/entry/koko/87f6c894a9814729bdf2af27104da93c  
...                                                   ...  
밤       10  #/entry/koko/89c9934457af4410ac482c1900ff24a0  
        11  #/entry/koko/89c9934457af4410ac482c1900ff24a0  
        12  #/entry/koko/b34c8705f6d546baaa3992ba844f3efd  
        13  #/entry/koko/27da9781e32a441db7936f91c46e6143  
        14  #/entry/koko/27da9781e32a441db7936f91c46e6143  

[90 rows x 2 columns]